In [1]:
import pandas as pd
import numpy as np
import re
import string
import pickle
import emoji

In [2]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text


def convert_emojis_to_words(text): 
    # Convert emojis to words using emoji.demojize
    text_with_emojis_as_words = emoji.demojize(text, delimiters=(' ', ' '))

    return text_with_emojis_as_words

In [3]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [4]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
def preprocessing(sentences):
    preprocessed_sentences = []

    for text in sentences:
        data = pd.DataFrame([text], columns=['Full_text'])
        data["Full_text"] = data["Full_text"].apply(lambda x: " ".join(x.lower() for x in x.split()))
        data["Full_text"] = data['Full_text'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
        data['Full_text'] = data['Full_text'].apply(convert_emojis_to_words)
        data["Full_text"] = data["Full_text"].apply(remove_punctuations)
        data["Full_text"] = data['Full_text'].str.replace('\d+', '', regex=True)
        data["Full_text"] = data["Full_text"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
        data["Full_text"] = data["Full_text"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
        preprocessed_sentences.append(data["Full_text"].iloc[0])

    return preprocessed_sentences
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new
def get_prediction(vectorized_text):
    vectorized_text = vectorized_text.reshape(1, -1)
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'positive'
    else:
        return 'negative'


In [5]:

sentences = ['amazing product', 'not expected with the quality', 'great experience']
preprocessed_sentences = preprocessing(sentences)

vectorized_sentences = vectorizer(preprocessed_sentences, tokens)

predictions = [get_prediction(vectorized_sentence) for vectorized_sentence in vectorized_sentences]

# Create a DataFrame
output_df = pd.DataFrame({'Sentence': sentences, 'Sentiment': predictions})

# Print the DataFrame
print(output_df)

                        Sentence Sentiment
0                amazing product  positive
1  not expected with the quality  negative
2               great experience  positive
